In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
from DQN import DQN
import numpy as np

# Hyper Parameters

In [31]:

BATCH_SIZE = 3000           # the previous memory size for learning
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 1000      # memory size
env_name = "MountainCar-v0"
env = gym.make(env_name)
env = env.unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape
print("Action:", N_ACTIONS)
print("States:", N_STATES)
print("action shape:", ENV_A_SHAPE)
dqn = DQN(BATCH_SIZE = BATCH_SIZE,
            LR = LR,
            EPSILON = EPSILON,    # gradient policy EPSILON present choose network
            GAMMA = GAMMA,        # the target blur level
            TARGET_REPLACE_ITER = TARGET_REPLACE_ITER,
            MEMORY_CAPACITY = MEMORY_CAPACITY,
            N_ACTIONS = N_ACTIONS,
            N_STATES = N_STATES,
            ENV_A_SHAPE = ENV_A_SHAPE)

Action: 3
States: 2
action shape: 0


# Train

In [ ]:
dqn = DQN(BATCH_SIZE = BATCH_SIZE,
            LR = LR,
            EPSILON = EPSILON,
            GAMMA = GAMMA,
            TARGET_REPLACE_ITER = TARGET_REPLACE_ITER,
            MEMORY_CAPACITY = MEMORY_CAPACITY,
            N_ACTIONS = N_ACTIONS,
            N_STATES = N_STATES,
            ENV_A_SHAPE = ENV_A_SHAPE)

# dqn.eval_net = torch.load("1.pkl").cuda()
print('\nCollecting experience...')
for i_episode in range(10):
    s = env.reset()
    ep_r = 0
    while True:
        env.render()
        a = dqn.choose_action(s)
        s_, r, done, info = env.step(a)
        
#         "CartPole-v1"
#         x, x_dot, theta, theta_dot = s_
#         r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
#         r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
#         r = r1 + r2
        
        dqn.store_transition(s, a, r, s_)
        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            if dqn.memory_counter == MEMORY_CAPACITY+1:
                print("-------- learning --------")
            dqn.learn()
        if done:
            break
        s = s_
    print('Ep: ', i_episode,
          '| Ep_r: ', round(ep_r, 2))
    if ep_r>400:
        break
dqn.save_model(env_name)


-------- learning --------
Ep:  0 | Ep_r:  -2421.0
Ep:  1 | Ep_r:  -1480.0


# TEST

In [ ]:
def choose_action(s):
    x = torch.unsqueeze(torch.FloatTensor(s), 0)
    # input only one sample
    if np.random.uniform() < dqn.EPSILON:   # greedy
        actions_value = dqn.eval_net.forward(x.cuda())
        action = torch.max(actions_value, 1)[1].cpu().data.numpy()
        action = action[0] if dqn.ENV_A_SHAPE == 0 else action.reshape(dqn.ENV_A_SHAPE)  # return the argmax index
    else:   # random
        action = np.random.randint(0, dqn.N_ACTIONS)
        action = action if dqn.ENV_A_SHAPE == 0 else action.reshape(dqn.ENV_A_SHAPE)
    return action

dqn = DQN(BATCH_SIZE = BATCH_SIZE,
            LR = LR,
            EPSILON = 0.5,
            GAMMA = GAMMA,
            TARGET_REPLACE_ITER = TARGET_REPLACE_ITER,
            MEMORY_CAPACITY = MEMORY_CAPACITY,
            N_ACTIONS = N_ACTIONS,
            N_STATES = N_STATES,
            ENV_A_SHAPE = ENV_A_SHAPE)
dqn.eval_net = torch.load(env_name+".pkl").cuda()

s=env.reset()
ep_r = 0
while True:
    env.render()
    a = choose_action(s)
    dqn.EPSILON += 0.0012   # dynamical adjust greedy
    s_, r, done, info = env.step(a)
    ep_r += r
    if done:
        break
    s = s_
print('Ep_r: ', round(ep_r, 2))

# Close environment

In [30]:
env.close()

In [15]:
 np.random.uniform()

0.8277021746315054